# Introduction and Set-up

Gradient-weighted class activation mapping is a great way to better understand what's happening in your CNN. It helps visualize what the important parts of your image are for classification.

In this notebook, we're going to be using Grad-CAM to see what parts of the image are important for the CNN when classifying between COVID-19 and pneumonia X-rays. From an initial analysis, it may  seem that COVID-19 and pneumonia images will be relatively similar, given than many cases of COVID-19 cause pnuemonia. The Grad-CAM visualizations may help pinpoint what differentiates these classes of images. 

Before we run our notebook, make sure to change the accelerator to TPU for quick training.

In [2]:
import re
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH = KaggleDatasets().get_gcs_path("covid19-radiography-database")
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = [180, 180]

# Load the images

In [ ]:
filenames = tf.io.gfile.glob(str(GCS_PATH + '/COVID-19 Radiography Database/COVID-19/*'))
filenames.extend(tf.io.gfile.glob(str(GCS_PATH + '/COVID-19 Radiography Database/NORMAL/*')))
filenames.extend(tf.io.gfile.glob(str(GCS_PATH + '/COVID-19 Radiography Database/Viral Pneumonia/*')))

random.seed(1337)
tf.random.set_seed(1337)
random.shuffle(filenames)

Divide the set into training, validation, and testing sets.

In [ ]:
train_filenames, test_filenames = train_test_split(filenames, test_size=0.1)
train_filenames, val_filenames = train_test_split(train_filenames, test_size=0.1)

In [ ]:
COUNT_NORMAL = len([filename for filename in train_filenames if "NORMAL" in filename])
print("Normal images count in training set: " + str(COUNT_NORMAL))

COUNT_COVID = len([filename for filename in train_filenames if "/COVID-19/" in filename])
print("COVID-19 images count in training set: " + str(COUNT_COVID))

COUNT_PNEUMONIA = len([filename for filename in train_filenames if "Viral" in filename])
print("Pneumonia images count in training set: " + str(COUNT_PNEUMONIA))

Normal images count in training set: 1080
COVID-19 images count in training set: 183
Pneumonia images count in training set: 1089


In [ ]:
train_list_ds = tf.data.Dataset.from_tensor_slices(train_filenames)
val_list_ds = tf.data.Dataset.from_tensor_slices(val_filenames)
test_list_ds = tf.data.Dataset.from_tensor_slices(test_filenames)

In [ ]:
TRAIN_IMG_COUNT = tf.data.experimental.cardinality(train_list_ds).numpy()
print("Training images count: " + str(TRAIN_IMG_COUNT))

VAL_IMG_COUNT = tf.data.experimental.cardinality(val_list_ds).numpy()
print("Validating images count: " + str(VAL_IMG_COUNT))

Training images count: 2352
Validating images count: 262


The following functions will help us format our dataset into the necessary (image, label) tuple for easy training. We also one-hot encode our labels (i.e. [1 0 0] means NORMAL).

In [ ]:
CLASSES = ['NORMAL', 'COVID-19', 'Viral Pneumonia']

In [ ]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    return parts[-2] == CLASSES

In [ ]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_png(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, IMAGE_SIZE)

In [ ]:
def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [ ]:
train_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
def prepare_for_training(ds, cache=True):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(BATCH_SIZE)

    if cache:
        ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

In [ ]:
train_ds = prepare_for_training(train_ds)
val_ds = prepare_for_training(val_ds)
test_ds = prepare_for_training(test_ds, False)

In [ ]:
with strategy.scope():
    reconstructed_model = keras.models.load_model("../input/test-model/xray_model.h5")
    reconstructed_model.pop()
    reconstructed_model.add(keras.layers.Dense(3, activation='softmax'))
    
    METRICS = [
        'accuracy',
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall")
    ]
    
    reconstructed_model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=METRICS,
    )

In [ ]:
history = reconstructed_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stopping_cb]
)

Epoch 1/20
19/19 [==============================] - 60s 3s/step - precision: 0.6982 - loss: 0.8479 - accuracy: 0.6637 - recall: 0.5765 - val_precision: 0.8584 - val_loss: 0.4569 - val_accuracy: 0.8244 - val_recall: 0.7634
Epoch 2/20
19/19 [==============================] - 2s 94ms/step - precision: 0.8667 - loss: 0.4890 - accuracy: 0.8448 - recall: 0.8155 - val_precision: 0.8867 - val_loss: 0.3025 - val_accuracy: 0.8740 - val_recall: 0.8664
Epoch 3/20
19/19 [==============================] - 2s 93ms/step - precision: 0.8819 - loss: 0.4007 - accuracy: 0.8686 - recall: 0.8542 - val_precision: 0.9134 - val_loss: 0.3163 - val_accuracy: 0.8969 - val_recall: 0.8855
Epoch 4/20
19/19 [==============================] - 2s 92ms/step - precision: 0.8925 - loss: 0.3470 - accuracy: 0.8831 - recall: 0.8754 - val_precision: 0.9186 - val_loss: 0.2319 - val_accuracy: 0.9122 - val_recall: 0.9046
Epoch 5/20
19/19 [==============================] - 2s 91ms/step - precision: 0.9101 - loss: 0.3114 - accurac

# Evaluate our model

In [ ]:
reconstructed_model.evaluate(test_ds, return_dict=True)

3/3 [==============================] - 4s 1s/step - precision: 0.9689 - loss: 0.1150 - accuracy: 0.9622 - recall: 0.9622


{'precision': 0.9688581824302673,
 'loss': 0.11499261111021042,
 'accuracy': 0.962199330329895,
 'recall': 0.962199330329895}

We correctly classify all of our testing images as well! Even though we had a very limited number of images, we could build a great model by loading in a pre-trained model.

# Grad-CAM Setup

Check out Keras IO [Grad-CAM Code Example](https://keras.io/examples/vision/grad_cam/) for the original source code.

To start, let's first look at the structure of our model.

In [23]:
model = keras.models.load_model("/content/classify_model4 (1).h5")

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 178, 178, 100)     2800      
                                                                 
 batch_normalization (BatchN  (None, 178, 178, 100)    400       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 100)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 70)        63070     
                                                                 
 batch_normalization_1 (Batc  (None, 57, 57, 70)       280       
 hNormalization)                                                 
                                                        

We need to get the output of the last convolution layer. Because I used Sequential instead of Functional API for my pneumonia model in my other notebook, the blocks that I used show us as nested Sequential models instead of as individual layers. That's not a problem because we can look at the structures of internal models as well.

In [29]:
# last convolution block of the model
model.layers[14].layers

AttributeError: ignored

Define the function to get a NumPy repressentation of our image.

In [14]:
def get_img_array(img_path, size=180):
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 NumPy array
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 180, 180, 3)
    array = np.expand_dims(array, axis=0) / 255.0
    return array

# Make grad-CAM heatmap

Let's define our grad-CAM function. We need to identify our last convolution layer. For our model, this would be the convolution layer within our `sequential_3` model in our summary above. Since we only need the outputs of the layer and not the actual layer itself, we can specify `sequential_3`, our 7th layer in oour model, as the last convolution layer.

We also need to specifiy our classifying layers. The flatten layer and the layers that follow it does the classifying for us so we'll label `layers[-5:]` as our classifying layers.

In [15]:
def make_gradcam_heatmap(img_array, model):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.layers[7]
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)
    
    # Mark the classifying layers
    classifier_layers = model.layers[-5:]

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for classifier_layer in classifier_layers:
        x = classifier_layer(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

# Define superimposing function

Let's superimpose the heatmap on the original image to visualize what the CNN marks as important and is used to classify the image.

In [16]:
def superimposed_cam(file_path):
    # Prepare image
    img_array = get_img_array(file_path)

    # Generate class activation heatmap
    heatmap = make_gradcam_heatmap(
        img_array, reconstructed_model
    )

    # Rescale the original image
    img = img_array * 255

    # We rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # We use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # We use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # We create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.4 + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img[0])
    
    return superimposed_img, CLASSES[np.argmax(reconstructed_model.predict(img_array))]

# Visualize class activation mapping

Let's compare what the model uses to classify COVID-19 X-rays versus Pneumonia X-rays.

In [17]:
!pip install --upgrade -q kaggle

!mkdir /root/.kaggle
import json
token = {
    "username": "zahidhussain909",
    "key": "39a06efd89d0f2a699143b8d3d62b216"
}

with open('/root/.kaggle/kaggle.json', 'w') as config_file:
    json.dump(token, config_file)
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d zahidhussain909/denoised-oct-balanced


 99% 1.04G/1.05G [00:08<00:00, 131MB/s]
100% 1.05G/1.05G [00:08<00:00, 134MB/s]


In [18]:
import zipfile
zipref=zipfile.ZipFile("/content/denoised-oct-balanced.zip",'r')
zipref.extractall()
zipref.close()

!rm -rf /content/denoised-oct-balanced.zip

In [19]:
covid_filenames = tf.io.gfile.glob('/content/DENOISED OCT/train/CNV/*')
pneumonia_filenames = tf.io.gfile.glob('/content/DENOISED OCT/train/NORMAL/*')

In [22]:


img, pred = superimposed_cam('/content/DENOISED OCT/train/NORMAL/10.png')
plt.imshow(img)
plt.title(pred)
plt.axis("off")


TypeError: ignored

The top two rows are COVID-19 images and the botton two rows are Pneumonia images. Parts of the image that are redder on the rainbow spectrum are the more "important" parts of the image, as defined by the CNN, and purple parts of the image are less important.

For this set of 10 COVID-19 images, it seems that our model focuses on one lung more than the other. This may be biologically significant, or it may be an artificial artifact of the model. One of the hardest aspects of computational biology is maneuvering the differences between biological vs computational significance. Having collaborative discussions with experts in both fields will help answer some of these questions. For the Pnueumonia images, at least for the 10 images shown here, it seems that images are seen in a more holistic manner than the COVID-19 images by the model. However, we've only displayed 20 images here. Looking at the rest of the images will give us a clearer picture. 